In [1]:
%config IPCompleter.greedy=True

In [2]:
import pandas as pd

df_train = pd.read_csv('train_kaggle.csv')
df_train

,Id,label
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
5,5,0
6,6,0
7,7,0
8,8,1
9,9,0


In [3]:
import pandas as pd
def load_features(col):
    return pd.read_parquet("filtered_features/filtered_feature{}.gzip".format(col))

In [4]:
import pandas as pd

def load_test_features(col):
    return pd.read_parquet("filtered_test_features/filtered_test_feature{}.gzip".format(col))

In [5]:
NUM_OF_COL = 40

In [6]:
SkipCol = {}
SkipCol[18] = True
SkipCol

{18: True}

In [7]:
y = df_train['label']
y = y.values
y

array([0, 0, 0, ..., 1, 0, 1], dtype=int64)

In [53]:
from sklearn import decomposition
import numpy as np

XTrainAll = []
for col in range(NUM_OF_COL):
    if col in SkipCol:
        continue
    tf = load_features(col)
    XTrainAll.append(tf)
XTrainAll = np.concatenate(XTrainAll,  axis=1)
XTrainAll.shape

(30336, 14136)

In [54]:
np.save('XTrainAll.npy', XTrainAll)

In [55]:
XTestAll = []
for col in range(NUM_OF_COL):
    if col in SkipCol:
        continue
    tf = load_test_features(col)
    tf = tf.replace([np.inf], np.finfo('float32').max).replace([np.inf, -np.inf], np.finfo('float32').min).fillna(0)
    XTestAll.append(tf)
XTestAll = np.concatenate(XTestAll,  axis=1)
XTestAll.shape

(10000, 14136)

In [56]:
np.save('XTestAll.npy', XTestAll)

In [18]:
df_test = pd.read_csv('sample_solution.csv')


In [58]:
y.shape

(30336,)

In [59]:
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier

clf = ExtraTreesClassifier(n_estimators=50)
# clf = GradientBoostingClassifier()
clf = clf.fit(XTrainAll, y)
modelSelection = SelectFromModel(clf, prefit=True, max_features=8000)
XTrain = modelSelection.transform(XTrainAll)
XTrain.shape

(30336, 3248)

In [60]:
np.save('XTrain.npy', XTrain)

In [10]:
import numpy as np
XTrain = np.load('XTrain.npy', allow_pickle=True)
XTrain.shape

(30336, 3248)

In [61]:
XTest = modelSelection.transform(XTestAll)
XTest.shape

(10000, 3248)

In [62]:
np.save('XTest.npy', XTest)

In [11]:
XTest = np.load('XTest.npy', allow_pickle=True)
XTest.shape

(10000, 3248)

In [ ]:
from lightgbm import LGBMModel
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt

clf = LGBMModel()
parameters = {'learning_rate': sp_randFloat(),
                  'subsample'    : sp_randFloat(),
                  'n_estimators' : sp_randInt(100, 1000),
                  'max_depth'    : sp_randInt(4, 10) 
                 }

randm = RandomizedSearchCV(estimator=gb_clf, param_distributions = parameters, 
                           cv = 5, n_iter = 10, n_jobs=-1)
randm.fit(XTrain, y)

# Results from Random Search
print("\n========================================================")
print(" Results from Random Search " )
print("========================================================")    

print("\n The best estimator across ALL searched params:\n",
      randm.best_estimator_)

print("\n The best score across ALL searched params:\n",
      randm.best_score_)

print("\n The best parameters across ALL searched params:\n",
      randm.best_params_)

print("\n ========================================================")

gb_clf = randm.best_estimator_

In [14]:
from lightgbm import LGBMClassifier

gb_clf = LGBMClassifier()
gb_clf.fit(XTrain, y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [15]:
probs = gb_clf.predict_proba(XTest)
YTest = probs[:, 0]
YTest

array([0.99710675, 0.99656644, 0.98467174, ..., 0.99714569, 0.99830632,
       0.9989995 ])

In [16]:
np.max(YTest)

0.999581969138716

In [19]:
df_test['Predicted'] = YTest
df_test

,Id,Predicted
0,0,0.997107
1,1,0.996566
2,2,0.984672
3,3,0.996327
4,4,0.996763
5,5,0.978892
6,6,0.989070
7,7,0.917925
8,8,0.998916
9,9,0.997564


In [20]:
df_test.to_csv('test.csv', index=False)